In [31]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.python.framework import ops

In [32]:
gHidden1OutputSize = 20
pHidden1OutputSize = 50
sHidden1OutputSize = 50
hidden2OutputSize = 1
seed = 0
batchSize = 10

In [33]:
##MUST GET RANK OF EACH BOARD

In [24]:


inputFile = 'features.pckl'
with open(inputFile, 'rb') as fid:
    features = pickle.load(fid)
globalFeatures = np.array([feature['globalFeatures'] for feature in features], dtype='float32')
pieceFeatures  = np.array([feature['pieceFeatures'] for feature in features], dtype='float32')
squareFeatures = np.array([feature['squareFeatures'] for feature in features], dtype='float32')

globalLength= len(globalFeatures)
pieceLength = len(pieceFeatures)
squareLength = len(squareFeatures)

In [25]:
train_input_queue[0]

<tf.Tensor 'input_producer/Gather:0' shape=(19,) dtype=float32>

In [30]:
tf.reset_default_graph()

globalTensor = tf.convert_to_tensor(globalFeatures, dtype=tf.float32)
pieceTensor = tf.convert_to_tensor(pieceFeatures, dtype=tf.float32)
squareTensor = tf.convert_to_tensor(squareFeatures, dtype=tf.float32)

train_input_queue = tf.train.slice_input_producer(
                                    [globalFeatures, pieceFeatures, squareFeatures],
                                    shuffle=False)

globalBatch, pieceBatch, squareBatch = tf.train.batch(
                                    train_input_queue,
                                    #[globalFeatures, pieceFeatures, squareFeatures],
                                    batch_size=batchSize
                                    #,num_threads=1
                                    )

#global variables
with tf.variable_scope('GlobalVariables'):
   # globalFeats = tf.placeholder(tf.float32, shape=(batchSize, globalLength), name='GlobalFeatures')#1 bit (everything is really just floats)
    #global hidden layer
    gHidden1 = tf.layers.dense(globalBatch, gHidden1OutputSize, name='GlobalHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
    #testWeights = tf.get_variable("kernel")
#piece-centric variables
with tf.variable_scope('Piece-CentricVariables'):
    #pieceFeats = tf.placeholder(tf.float32, shape=(batchSize, pieceLength), name='Piece-CentricFeatures')

    #piece-centric hidden layer
    pHidden1 = tf.layers.dense(pieceBatch, pHidden1OutputSize, name='Piece-CentricHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))

#square-centric variables
with tf.variable_scope('Sqaure-CentricVariables'):
    #squareFeats = tf.placeholder(tf.float32, shape=(batchSize, squareLength), name='Square-CentricFeatures')

    #square-centric hidden layer
    sHidden1 = tf.layers.dense(squareBatch, sHidden1OutputSize, name='Square-CentricHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))

input2 = tf.concat([gHidden1, pHidden1, sHidden1], 1)
hidden2 = tf.layers.dense(input2, hidden2OutputSize, name='HiddenLayer2', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
prediction = tf.nn.sigmoid(hidden2)
loss = tf.losses.sigmoid_cross_entropy(prediction, groundTruth) #groundTruth must be of shape [batchSize, 1]
optimizer = tf.train.AdadeltaOptimizer()
train = optimizer.minimize(loss)

NameError: name 'groundTruth' is not defined

In [27]:
globalFeatures.shape


(9109, 19)

In [28]:


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #print sess.run( weights = tf.get_variable("GlobalVariable/kernel"))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for iter in range(5):
        print sess.run(hidden2)
    coord.request_stop()
    coord.join(threads)
    coord.request_stop()
    coord.join(threads)


[[1.363521  ]
 [1.4875631 ]
 [1.1470423 ]
 [0.4235489 ]
 [1.0428336 ]
 [0.80934733]
 [0.24529743]
 [0.87195814]
 [1.6052488 ]
 [1.1303871 ]]
[[1.3467827 ]
 [1.3130597 ]
 [1.3701539 ]
 [0.86757576]
 [0.99898285]
 [0.4736976 ]
 [0.98449576]
 [0.69551396]
 [0.8062087 ]
 [1.4974073 ]]
[[0.49809003]
 [1.4347339 ]
 [1.5448775 ]
 [0.5137142 ]
 [1.118883  ]
 [0.6082185 ]
 [0.48892057]
 [1.052182  ]
 [0.5482751 ]
 [1.0310432 ]]
[[1.044662  ]
 [1.2159138 ]
 [0.6246499 ]
 [1.2099564 ]
 [0.75562173]
 [0.4649952 ]
 [1.403624  ]
 [1.0682416 ]
 [0.41458696]
 [0.93178165]]
[[0.84872043]
 [0.6571884 ]
 [0.9392091 ]
 [0.635502  ]
 [0.75518763]
 [0.7238356 ]
 [1.1787624 ]
 [0.8004128 ]
 [1.1136863 ]
 [1.3335631 ]]


In [ ]:
globalTensor

In [ ]:
pieceFeatures